### Hyperparameter tuning with grid search

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from core.util.get_datasets import get_park_dataset
from sklearn.model_selection import ParameterGrid
from core.models.LSTM import LSTM
from core.models.model_training import train_model
from core.util.trefor_dataset import TreforData

Set ML model, loss function and hyperparameters that that will be tested.

In [12]:
param_grid = {
    "batch_size": [128],
    "hidden_size": [32, 64],
    "num_stacked_layers": [1, 2],
    "epochs": [10],
    "lookback": [24, 168],
}
model_used = LSTM
loss_function = nn.HuberLoss()

Use CUDA (GPU) if available.

In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

Train a model with specified hyperparameters

In [14]:
def train_with_params(params: dict) -> tuple[float, LSTM]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    hidden_size = params["hidden_size"]
    num_stacked_layers = params["num_stacked_layers"]
    epochs = params["epochs"]
    lookback = params["lookback"]

    # Load the data and split into training and validation sets
    x_train, y_train, x_val, y_val, _, _ = get_park_dataset(lookback, 24)
    x_train = torch.tensor(x_train).float()
    y_train = torch.tensor(y_train).float()
    x_val = torch.tensor(x_val).float()
    y_val = torch.tensor(y_val).float()
    train_dataset = TreforData(x_train, y_train, device)
    val_dataset = TreforData(x_val, y_val, device)

    # Create data loaders
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = model_used(
        input_size=x_train.shape[-1],
        hidden_size=hidden_size,
        num_stacked_layers=num_stacked_layers,
        dropout_rate=0.2,
    ).to(device)

    _, val_loss, best_model = train_model(
        epochs, model, loss_function, training_loader, validation_loader
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [ ]:
best_loss = float("inf")
best_params = None
best_model = None

for params in ParameterGrid(param_grid):
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    if v_loss < best_loss:
        best_loss = v_loss
        best_params = params
        best_model = model

print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)